## Edgeに自動的にclickする

In [176]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np

url = 'https://www.amazon.co.jp/'
options=webdriver.EdgeOptions()
options.add_argument('--headless')
driver = webdriver.Edge(options=options)
driver.get(url)

find = driver.find_elements(by='id', value='twotabsearchtextbox')
find[0].send_keys(input('ban muon mua sach gi'))
find[0].send_keys(Keys.RETURN )

In [177]:
page_source=BeautifulSoup(driver.page_source,'html.parser')
profiles=page_source.find_all('a',class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')

all_profiles=[]
for profile in profiles:
    profile_id=profile.get('href')
    profile_url='https://www.amazon.co.jp'+profile_id
    if profile_url not in all_profiles:
        all_profiles.append(profile_url)
print(all_profiles)


['https://www.amazon.co.jp/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A3QRZMS7FODE7V&qualifier=1684117846&id=3467903299615544&widgetName=sp_atf&url=%2F%25E6%259A%25AE%25E3%2582%2589%25E3%2581%2597%25E3%2581%258C%25E5%25A4%2589%25E3%2582%258F%25E3%2582%258B%25E6%2599%2582%25E7%259F%25AD%25E5%25AE%25B6%25E4%25BA%258B-%25E5%25AE%25B6%25E4%25BA%258B%25E3%2580%2581%25E8%2582%25B2%25E5%2585%2590%25E3%2580%2581%25E4%25BB%2595%25E4%25BA%258B%25E3%2581%25A7%25E3%2583%259C%25E3%2583%25AD%25E3%2583%259C%25E3%2583%25AD%25E3%2581%25A0%25E3%2581%25A3%25E3%2581%259F%25E7%25A7%2581%25E3%2581%258C20%25E6%2599%2582%25E4%25BB%25A5%25E9%2599%258D%25E3%2582%2592%25E8%2587%25AA%25E5%2588%2586%25E6%2599%2582%25E9%2596%2593%25E3%2581%25AB%25E3%2581%2599%25E3%2582%258B%25E3%2581%25BE%25E3%2581%25A7-%25E3%2581%258B%25E3%2581%258A%25E3%2582%258A-ebook%2Fdp%2FB098WMCYBT%2Fref%3Dsr_1_1_sspa%3F__mk_ja_JP%3D%25E3%2582%25AB%25E3%2582%25BF%25E3%2582%25AB%25E3%2583%258A%26keywords%3D%25E

## linkを取り使う

In [178]:

def get_url():
    page_source=BeautifulSoup(driver.page_source,'html.parser')
    profiles=page_source.find_all('a',class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    
    all_profiles=[]
    for profile in profiles:
        profile_id=profile.get('href')
        profile_url='https://www.amazon.co.jp'+profile_id
        if profile_url not in all_profiles:
            all_profiles.append(profile_url)
    return all_profiles



In [179]:
def scrape_data():
    d={'id':[],"name":[],'price':[],"rating":[],"comment":[]}
    for link in get_url():
        driver.get(link)
        d['id'].append(link)
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        try:
            name = page_source.find('span', id='productTitle').text.strip()
            d['name'].append(name)
        except:
                 d['name'].append("")
        try:
            price = page_source.find('span', class_='a-price-whole').text.strip()
            d['price'].append(price)

        except:
             d['price'].append("")    
        try:
            rating = page_source.find('span', class_='a-icon-alt').text.strip()
            d['rating'].append(rating)
        except:
             d['rating'].append("")
        comment = page_source.find('span',id="acrCustomerReviewText")
        if comment is not None:
            d['comment'].append(comment.text.strip())
        else:
            d['comment'].append('')
        print('\n')
    
    return d


In [ ]:

df=pd.DataFrame.from_dict(scrape_data())
df['price'].replace("",np.nan,inplace=True)
df=df.dropna(subset=['price'])
df['price']=df['price'].str.replace(',',"").astype(int)


In [181]:
df['comment'].replace("",np.nan,inplace=True)
df=df.dropna(subset=['comment'])


## 最高価格と最低価格は何ですか？

In [182]:
valuse=df.groupby('name')['price'].sum()
valuse_sorted=valuse.sort_values(ascending=False)
max=valuse_sorted.iloc[0]
min=valuse_sorted.iloc[-1]
print("最高価格は '{}'という商品 {}円があります、\n逆に最低価格は ’{}’という商品 {}円があります".format(valuse_sorted.index[0],max,valuse_sorted.index[-1],min))


最高価格は '家事事件手続法下における書記官事務の運用に関する実証的研究―別表第一事件を中心に―'という商品 8800円があります、
逆に最低価格は ’自然と片づく部屋になる！ 長続きするラク家事収納術’という商品 494円があります


## 評価（rating）が最も多い星数の商品数と最も少ない星数の商品数

In [243]:
df['rating'].str.slice(-3,).astype(float)
valuse_rating=df.groupby('name')['rating'].sum()
sorted_valuse_rating=valuse_rating.sort_values(ascending=False)
max_rating=sorted_valuse_rating.iloc[0]
min_rating=sorted_valuse_rating.iloc[-1]
print("評価が最も多い星数の商品数は{}というの商品が{}です、\n逆に最も少ない星数の商品数は{}という商品が{}です".format(sorted_valuse_rating.index[0],max_rating,sorted_valuse_rating.index[-1],min_rating))

評価が最も多い星数の商品数は“当事者に寄り添う"家事調停委員の基本姿勢と実践技術というの商品が5つ星のうち5.0です、
逆に最も少ない星数の商品数は別冊エッセ　捨ててやめてラクになる　やめる家事　コンパクト版という商品が5つ星のうち3.0です


## 最も多くのレビューを持つ商品と最も少ないのレビューを持つ商品

In [198]:
df['comment_count']=df['comment'].str.replace("個の評価","")
valuse_commet=df.groupby('name')['comment_count'].sum()
sorte_valuse_commet=valuse_commet.sort_values(ascending=False)
max_commet=sorte_valuse_commet.iloc[0]
min_commet=sorte_valuse_commet.iloc[-1]
print('最も多くのレビューを持つ商品は{}という商品が{}個あります、\n最も少ないのレビューを持つ商品は{}という商品が{}個あります'.format(sorte_valuse_commet.index[0],max_commet,sorte_valuse_commet.index[-1],min_commet))

最も多くのレビューを持つ商品は「家事のしすぎ」が日本を滅ぼす (光文社新書)という商品が90個あります、
最も少ないのレビューを持つ商品は西園寺さんは家事をしない（２） (BE・LOVEコミックス)という商品が1個あります


## 産品名が重複している製品

In [226]:
duplicate_count = len(df[df.duplicated(subset=['name'], keep=False)].value_counts())


産品名が重複している製品は4個ありますか


In [238]:
unique_names = df['name'].unique()

for name in unique_names:
    count = df[df['name'] == name]['name'].count()
    if count > 1:
       print(f'産品名が重複している製品は{name}です。')



産品名が重複している製品はクロワッサン特別編集 無駄のない「家事」。 (マガジンハウスムック)です。
産品名が重複している製品はアラフォーまきこのごゆるり家事 (コミックエッセイ)です。


## 2000円以上で評価が最高の製品は何個ありますか

In [270]:

df['price'] = df['price'].astype(float)
top_products = df[(df['price'] > 2000) & (df["rating"] >"4.5")]
counts=len(top_products)
print(f"2000円以上で評価が最高の製品は{count}個あります")
print('それは：')
top_products[['name']]

2000円以上で評価が最高の製品は3個あります
それは：


,name
27,"“当事者に寄り添う""家事調停委員の基本姿勢と実践技術"
36,ノン子36歳(家事手伝い) [DVD]
47,家事事件手続法下における書記官事務の運用に関する実証的研究―別表第一事件を中心に―
